In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import ensemble
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import datetime


In [4]:
df = pd.read_csv('nyc bus data.csv')
df = df.drop_duplicates()
# = df.dropna()
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,40.635170,-73.960803,FOSTER AV/E 18 ST,approaching,76,6/1/2017 0:03,24:06:14,NaN
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,40.590802,-74.158340,MERRYMOUNT ST/TRAVIS AV,approaching,62,6/1/2017 0:03,23:58:02,NaN
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,40.886010,-73.912647,HENRY HUDSON PKY E/W 235 ST,at stop,5,6/1/2017 0:03,24:00:53,NaN
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,40.668002,-73.729348,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267,6/1/2017 0:04,24:03:00,NaN
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,40.868134,-73.893032,GRAND CONCOURSE/E 196 ST,at stop,11,6/1/2017 0:03,23:59:38,NaN


In [5]:
df.shape

(1048575, 18)

In [6]:
#subset the dataframe to include only the first 100,000 rows, or observations.
df = df.iloc[:100000, :]
df.shape

(100000, 18)

In [7]:
df.isnull().sum()

RecordedAtTime                   0
DirectionRef                     0
PublishedLineName                0
OriginName                       0
OriginLat                        0
OriginLong                       0
DestinationName                  0
DestinationLat                   0
DestinationLong                  0
VehicleRef                       0
VehicleLocation.Latitude         0
VehicleLocation.Longitude        0
NextStopPointName               96
ArrivalProximityText            96
DistanceFromStop                96
ExpectedArrivalTime          11370
ScheduledArrivalTime          1688
Unnamed: 17                  99995
dtype: int64

In [10]:
df.columns

Index(['RecordedAtTime', 'DirectionRef', 'PublishedLineName', 'OriginName',
       'OriginLat', 'OriginLong', 'DestinationName', 'DestinationLat',
       'DestinationLong', 'VehicleRef', 'VehicleLocation.Latitude',
       'VehicleLocation.Longitude', 'NextStopPointName',
       'ArrivalProximityText', 'DistanceFromStop', 'ExpectedArrivalTime',
       'ScheduledArrivalTime', 'Unnamed: 17'],
      dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 18 columns):
RecordedAtTime               100000 non-null object
DirectionRef                 100000 non-null int64
PublishedLineName            100000 non-null object
OriginName                   100000 non-null object
OriginLat                    100000 non-null float64
OriginLong                   100000 non-null float64
DestinationName              100000 non-null object
DestinationLat               100000 non-null float64
DestinationLong              100000 non-null float64
VehicleRef                   100000 non-null object
VehicleLocation.Latitude     100000 non-null float64
VehicleLocation.Longitude    100000 non-null float64
NextStopPointName            99904 non-null object
ArrivalProximityText         99904 non-null object
DistanceFromStop             99904 non-null object
ExpectedArrivalTime          88630 non-null object
ScheduledArrivalTime         98312 non-null object
Unname

In [16]:
#converting strings to datetimes
pd.to_datetime(df['ExpectedArrivalTime'], errors="coerce")

0       2017-06-01 00:03:00
1       2017-06-01 00:03:00
2       2017-06-01 00:03:00
3       2017-06-01 00:04:00
4       2017-06-01 00:03:00
5       2017-06-01 00:03:00
6       2017-06-01 00:03:00
7       2017-06-01 00:04:00
8       2017-06-01 00:03:00
9       2017-06-01 00:03:00
10      2017-06-01 00:04:00
11      2017-06-01 00:03:00
12      2017-06-01 00:03:00
13      2017-06-01 00:04:00
14      2017-06-01 00:04:00
15      2017-06-01 00:03:00
16      2017-06-01 00:04:00
17      2017-06-01 00:04:00
18      2017-06-01 00:03:00
19      2017-06-01 00:04:00
20      2017-06-01 00:03:00
21      2017-06-01 00:03:00
22      2017-06-01 00:04:00
23      2017-06-01 00:03:00
24      2017-06-01 00:06:00
25      2017-06-01 00:03:00
26      2017-06-01 00:04:00
27      2017-06-01 00:03:00
28      2017-06-01 00:03:00
29      2017-06-01 00:04:00
                ...        
99970   2017-06-01 12:04:00
99971   2017-06-01 12:04:00
99972   2017-06-01 12:04:00
99973   2017-06-01 12:04:00
99974   2017-06-01 1

In [33]:
#df['ExpectedArrivalTime'].isna(), this didnt work in this case

print((df['ExpectedArrivalTime'] == pd.NaT).sum())

0


In [14]:
#converting strings to datetimes.
pd.to_datetime(df['ScheduledArrivalTime'], errors="coerce")

0                       NaT
1       2019-04-11 23:58:02
2                       NaT
3                       NaT
4       2019-04-11 23:59:38
5                       NaT
6                       NaT
7       2019-04-11 23:58:47
8                       NaT
9                       NaT
10      2019-04-11 23:48:35
11                      NaT
12                      NaT
13                      NaT
14                      NaT
15                      NaT
16                      NaT
17                      NaT
18                      NaT
19      2019-04-11 23:46:00
20                      NaT
21                      NaT
22      2019-04-11 23:55:00
23                      NaT
24                      NaT
25                      NaT
26                      NaT
27      2019-04-11 23:57:00
28                      NaT
29      2019-04-11 23:52:51
                ...        
99970   2019-04-11 11:55:40
99971   2019-04-11 11:58:45
99972   2019-04-11 11:46:06
99973   2019-04-11 12:07:33
99974   2019-04-11 1

In [38]:
print(df['ScheduledArrivalTime'] == pd.NaT)

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
99970    False
99971    False
99972    False
99973    False
99974    False
99975    False
99976    False
99977    False
99978    False
99979    False
99980    False
99981    False
99982    False
99983    False
99984    False
99985    False
99986    False
99987    False
99988    False
99989    False
99990    False
99991    False
99992    False
99993    False
99994    False
99995    False
99996    False
99997    False
99998    False
99999    False
Name: ScheduledArrivalTime, Length: 100000, dtype: bool


In [15]:
df['ExpectedArrivalTime']

0         6/1/2017 0:03
1         6/1/2017 0:03
2         6/1/2017 0:03
3         6/1/2017 0:04
4         6/1/2017 0:03
5         6/1/2017 0:03
6         6/1/2017 0:03
7         6/1/2017 0:04
8         6/1/2017 0:03
9         6/1/2017 0:03
10        6/1/2017 0:04
11        6/1/2017 0:03
12        6/1/2017 0:03
13        6/1/2017 0:04
14        6/1/2017 0:04
15        6/1/2017 0:03
16        6/1/2017 0:04
17        6/1/2017 0:04
18        6/1/2017 0:03
19        6/1/2017 0:04
20        6/1/2017 0:03
21        6/1/2017 0:03
22        6/1/2017 0:04
23        6/1/2017 0:03
24        6/1/2017 0:06
25        6/1/2017 0:03
26        6/1/2017 0:04
27        6/1/2017 0:03
28        6/1/2017 0:03
29        6/1/2017 0:04
              ...      
99970    6/1/2017 12:04
99971    6/1/2017 12:04
99972    6/1/2017 12:04
99973    6/1/2017 12:04
99974    6/1/2017 12:04
99975    6/1/2017 12:04
99976    6/1/2017 12:04
99977    6/1/2017 12:04
99978    6/1/2017 12:05
99979    6/1/2017 12:04
99980    6/1/201

In [10]:
#filling in missing data 
df['ExpectedArrivalTime'].ffill(axis=0, inplace=True)
df['ScheduledArrivalTime'].ffill(axis=0, inplace=True)
df['DistanceFromStop'].ffill(axis=0, inplace=True)
df['ArrivalProximityText'].ffill(axis=0, inplace=True)
df['NextStopPointName'].ffill(axis=0, inplace=True)

In [11]:
df.head()

,RecordedAtTime,DirectionRef,PublishedLineName,OriginName,OriginLat,OriginLong,DestinationName,DestinationLat,DestinationLong,VehicleRef,VehicleLocation.Latitude,VehicleLocation.Longitude,NextStopPointName,ArrivalProximityText,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,Unnamed: 17
0,6/1/2017 0:03,0,B8,4 AV/95 ST,40.616104,-74.031143,BROWNSVILLE ROCKAWAY AV,40.656048,-73.907379,NYCT_430,40.635170,-73.960803,FOSTER AV/E 18 ST,approaching,76,6/1/2017 0:03,24:06:14,NaN
1,6/1/2017 0:03,1,S61,ST GEORGE FERRY/S61 & S91,40.643169,-74.073494,S I MALL YUKON AV,40.575935,-74.167686,NYCT_8263,40.590802,-74.158340,MERRYMOUNT ST/TRAVIS AV,approaching,62,6/1/2017 0:03,23:58:02,NaN
2,6/1/2017 0:03,0,Bx10,E 206 ST/BAINBRIDGE AV,40.875008,-73.880142,RIVERDALE 263 ST,40.912376,-73.902534,NYCT_4223,40.886010,-73.912647,HENRY HUDSON PKY E/W 235 ST,at stop,5,6/1/2017 0:03,24:00:53,NaN
3,6/1/2017 0:03,0,Q5,TEARDROP/LAYOVER,40.701748,-73.802399,ROSEDALE LIRR STA via MERRICK,40.666012,-73.735939,NYCT_8422,40.668002,-73.729348,HOOK CREEK BL/SUNRISE HY,< 1 stop away,267,6/1/2017 0:04,24:03:00,NaN
4,6/1/2017 0:03,1,Bx1,RIVERDALE AV/W 231 ST,40.881187,-73.909340,MOTT HAVEN 136 ST via CONCOURSE,40.809654,-73.928360,NYCT_4710,40.868134,-73.893032,GRAND CONCOURSE/E 196 ST,at stop,11,6/1/2017 0:03,23:59:38,NaN


In [12]:
#creating new feature column to indicate if a bus is on time or behind schedule
df['hour'] = df['ExpectedArrivalTime'].dt.hour
df['minute'] = df['ExpectedArrivalTime'].dt.minute

AttributeError: Can only use .dt accessor with datetimelike values

In [9]:
df.describe()

,DirectionRef,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.489260,40.722513,-73.934568,40.730404,-73.930072,40.725437,-73.932166
std,0.499887,0.095338,0.105286,0.088014,0.091688,0.088931,0.094125
min,0.000000,40.506882,-74.248062,40.508106,-74.246948,40.502898,-74.252328
25%,0.000000,40.645504,-73.989365,40.664642,-73.988831,40.653714,-73.980286
50%,0.000000,40.707813,-73.931328,40.716393,-73.932266,40.715176,-73.934826
75%,1.000000,40.809616,-73.878334,40.807545,-73.878326,40.804272,-73.880005
max,1.000000,40.912365,-73.701866,40.912376,-73.701454,40.912389,-73.701559
